**EXTRACCIÓN Y LIMPIEZA DE DATOS**

1. EXTRACCIÓN DE DATOS

In [1]:
# importación librarias

import pandas as pd


import numpy as np

import warnings
warnings.filterwarnings ('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import seaborn as sns

import matplotlib.pyplot as plt

In [2]:

from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import warnings
warnings.filterwarnings('ignore')
from selenium import webdriver
from selenium.webdriver.common.by import By

In [3]:
#driver configuration
opciones=Options()

opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.headless=False    # si True, no aperece la ventana (headless=no visible)
#opciones.add_argument('--start-maximized')         # comienza maximizado
#opciones.add_argument('user-data-dir=selenium')    # mantiene las cookies
#opciones.add_extension('driver_folder/adblock.crx')       # adblocker
#opciones.add_argument('--incognito')

1.1 Importación CSV de Base de datos de Registro de Servicios Sociales de la Comunidad de Madrid

 - **Fuente**: 'https://datos.comunidad.madrid/catalogo/dataset/servicios_sociales_registro_centros'
 - **Método de extracción**: descarga CSV de la página web oficial.

In [4]:
#import csv and creat a copy of the original df

ssmadrid_ori = pd.read_csv('../ETL-Project/data/servicios_sociales_registro_centros.csv',
                                  encoding = 'latin1', on_bad_lines='skip',sep=';')

ssmadrid = ssmadrid_ori.copy()

In [5]:
ssmadrid.head(1)

,numero_registro,denominacion,plazas_autorizadas_numero,plazas_descripcion,tipologia,subtipo,sector,municipio_codigo,municipio_nombre,direccion_via_tipo,direccion_via_codigo,direccion_via_nombre,direccion_portal,direccion_coor_x,direccion_coor_y,direccion_codigo_postal,contacto_telefono,entidad_denominacion,entidad_cif,entidad_naturaleza
0,C0003,CENTRO DE REHABILITACION LABORAL NUEVA VIDA,40,Capacidad máxima simultánea,CENTROS DE REHABILITACIÓN,CENTROS DE REHABILITACIÓN: OTROS,Personas con enfermedad mental crónica,079,Madrid,CALLE,01394,de Colomer,14,443535,4475980,28028,91355 26 80,ASOCIACION PSIQUIATRIA Y VIDA,G28727261,PRIVADA SIN ÁNIMO DE LUCRO


1.2 Scraping codigos postales de la comunidad de Madrid

- **Fuente**: 'https://www.worldpostalcodes.org/es/espana/comunidad-autonoma/lista-de-codigos-postales-en-community-of-madrid'
- **Método**: scraping utilizando la herramienta Selenium


In [6]:
PATH = ChromeDriverManager().install()     # instala el driver de chrome

driver=webdriver.Chrome(PATH, options=opciones)      # abre una venta una de chrome


In [7]:
url = 'https://www.worldpostalcodes.org/es/espana/comunidad-autonoma/lista-de-codigos-postales-en-community-of-madrid'

In [11]:
driver.implicitly_wait(3)
driver.get(url)

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=110.0.5481.78)
Stacktrace:
Backtrace:
	(No symbol) [0x007037D3]
	(No symbol) [0x00698B81]
	(No symbol) [0x0059B36D]
	(No symbol) [0x0057E5DA]
	(No symbol) [0x005EF0CB]
	(No symbol) [0x00600A56]
	(No symbol) [0x005EB216]
	(No symbol) [0x005C0D97]
	(No symbol) [0x005C253D]
	GetHandleVerifier [0x0097ABF2+2510930]
	GetHandleVerifier [0x009A8EC1+2700065]
	GetHandleVerifier [0x009AC86C+2714828]
	GetHandleVerifier [0x007B3480+645344]
	(No symbol) [0x006A0FD2]
	(No symbol) [0x006A6C68]
	(No symbol) [0x006A6D4B]
	(No symbol) [0x006B0D6B]
	BaseThreadInitThunk [0x75717D69+25]
	RtlInitializeExceptionChain [0x771BBB9B+107]
	RtlClearBits [0x771BBB1F+191]


In [10]:
tabla = driver.find_elements(By.XPATH, '/html/body/div/div/div/div/table[1]')

litatabla = [e.text.strip().split('\n') for e in tabla]




NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=110.0.5481.78)
Stacktrace:
Backtrace:
	(No symbol) [0x007037D3]
	(No symbol) [0x00698B81]
	(No symbol) [0x0059B36D]
	(No symbol) [0x0057E5DA]
	(No symbol) [0x005EF0CB]
	(No symbol) [0x00600A56]
	(No symbol) [0x005EB216]
	(No symbol) [0x005C0D97]
	(No symbol) [0x005C253D]
	GetHandleVerifier [0x0097ABF2+2510930]
	GetHandleVerifier [0x009A8EC1+2700065]
	GetHandleVerifier [0x009AC86C+2714828]
	GetHandleVerifier [0x007B3480+645344]
	(No symbol) [0x006A0FD2]
	(No symbol) [0x006A6C68]
	(No symbol) [0x006A6D4B]
	(No symbol) [0x006B0D6B]
	BaseThreadInitThunk [0x75717D69+25]
	RtlInitializeExceptionChain [0x771BBB9B+107]
	RtlClearBits [0x771BBB1F+191]


In [ ]:
columnas=driver.find_elements(By.TAG_NAME, 'th') 

listcols= [e.text.strip() for e in columnas]

listcols

In [ ]:
data = [[e.text for e in f.find_elements(By.TAG_NAME, 'td')]  
        for f in driver.find_element(By.TAG_NAME, 'tbody').find_elements(By.TAG_NAME, 'tr')]

data = data[1:]

In [ ]:
cp_madrid = pd.DataFrame(data, columns = listcols)

cp_madrid.head(1)

In [ ]:
cp_madrid.columns = [e.lower(). replace(' ', '_') for e in cp_madrid]

cp_madrid.head()

In [ ]:
cp_madrid.to_csv('../ETL-Project/data/cp_madrid.csv', index = False)

1.3 Población por cada código postal de la Comunidad de Madrid de 2022

- **Fuente**: INE = 'https://www.ine.es/jaxiT3/Tabla.htm?t=2881'
- **Método**: configuración de la selección de datos (población por cada CP de la Comunidad de Madrid del año 2022, por género y número total de personas)

1.4 Ayudas Sociales de la Comunidad de Madrid de 2022

- **Fuente**: Portal de datos abiertos del Ayuntamiento de Madrid = 'https://datos.madrid.es/portal/site/egob'
- **Método**: descarga de CSV

1.5 Panel de indicadores de distritos y barrios de Madrid de 2022.
- **Fuente**: Portal de datos abiertos del Ayuntamiento de Madrid = 'https://datos.madrid.es/portal/site/egob'
- **Método**: descarga de documento Excel.